<a href="https://colab.research.google.com/github/JAVINCHIE/third/blob/main/Binary_classification_with_stacking_ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e7:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F73291%2F8930475%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240720%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240720T201854Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D612356566aae7e99f00ba82df225856f34e85ae5b64f6b765d78e8abc7c5cbc6c76c084df266fe433520db4632cd647d0bdefe2cc7c1f1590bafa9137569d21e5daeaf356b025392c8785e5022d2c100ea7b6cfdf50daa24e38077cb0fabe7afec7b0f599032dd68ccfea0d8ba227bbda2d6c63edb83cc3eb3c44068d1676b549e643a28e0fd5ebfe7d30100b0b7f52bc58f4622fdf54405cae558829f09d26eb90a3aec84986f7b71683b385e476f990963a295a37304894bb912129660085002694c5d5c2dbb531b3a833584f4f5fe1e57a1abe977ca3774e33eaab97601234963ea21ba27059511a822f3c37a3df19548bb0aff388b98e5ef7242c5f2d75e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 259402906 bytes downloaded
Downloaded and uncompressed: playground-series-s4e7
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/test.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/sample_submission.csv


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')

df.head()



,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [5]:
#  Identify the columns of a data frame with missing values
missing_value_columns = df.columns[df.isnull().any()]
print("Columns with missing values:", missing_value_columns)



Columns with missing values: Index([], dtype='object')


In [6]:
# Check for duplicates
duplicates = df.duplicated()
print("Duplicate entries:", df[duplicates])

Duplicate entries: Empty DataFrame
Columns: [id, Gender, Age, Driving_License, Region_Code, Previously_Insured, Vehicle_Age, Vehicle_Damage, Annual_Premium, Policy_Sales_Channel, Vintage, Response]
Index: []


In [7]:
# Count the number of entries for each unique value of the specified attributes
gender_counts = df['Gender'].value_counts()
vehicle_age_counts = df['Vehicle_Age'].value_counts()
vehicle_damage_counts = df['Vehicle_Damage'].value_counts()

# Print the counts
print("Gender Counts:\n", gender_counts)
print("\nVehicle Age Counts:\n", vehicle_age_counts)
print("\nVehicle Damage Counts:\n", vehicle_damage_counts)

Gender Counts:
 Gender
Male      6228134
Female    5276664
Name: count, dtype: int64

Vehicle Age Counts:
 Vehicle_Age
1-2 Year     5982678
< 1 Year     5044145
> 2 Years     477975
Name: count, dtype: int64

Vehicle Damage Counts:
 Vehicle_Damage
Yes    5783229
No     5721569
Name: count, dtype: int64


In [8]:
# Function to calculate counts and percentages
def calculate_counts_and_percentages(df, column):
    counts = df[column].value_counts()
    percentages = df[column].value_counts(normalize=True) * 100
    return pd.DataFrame({'Count': counts, 'Percentage': percentages})

# Calculate counts and percentages for each specified attribute
gender_counts_and_percentages = calculate_counts_and_percentages(df, 'Gender')
vehicle_age_counts_and_percentages = calculate_counts_and_percentages(df, 'Vehicle_Age')
vehicle_damage_counts_and_percentages = calculate_counts_and_percentages(df, 'Vehicle_Damage')

# Print the results
print("Gender Counts and Percentages:\n", gender_counts_and_percentages)
print("\nVehicle Age Counts and Percentages:\n", vehicle_age_counts_and_percentages)
print("\nVehicle Damage Counts and Percentages:\n", vehicle_damage_counts_and_percentages)


Gender Counts and Percentages:
           Count  Percentage
Gender                     
Male    6228134   54.135101
Female  5276664   45.864899

Vehicle Age Counts and Percentages:
                Count  Percentage
Vehicle_Age                     
1-2 Year     5982678   52.001591
< 1 Year     5044145   43.843838
> 2 Years     477975    4.154571

Vehicle Damage Counts and Percentages:
                   Count  Percentage
Vehicle_Damage                     
Yes             5783229   50.267975
No              5721569   49.732025


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np


# Take a sample of the data to reduce size (adjust size as needed)
df_sample = df.sample(frac=0.1, random_state=42)  # 10% of the data

# Split the data into features and target (assuming 'Response' is the target column)
X = df_sample.drop(columns='Response')
y = df_sample['Response']

# Identify categorical and numerical columns
categorical_columns = X.select_dtypes(include=['object', 'category']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing pipeline for numerical data
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing pipeline for categorical data
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_columns),
        ('cat', categorical_pipeline, categorical_columns)
    ]
)

# Define the logistic regression model with a more efficient solver
logreg = LogisticRegression(solver='saga', max_iter=1000)  # Using 'saga' solver for large datasets

# Combine preprocessing steps and the model into a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', logreg)
])

# Define the parameter grid for random search
param_grid = {
    'classifier__C': np.logspace(-3, 3, 7),
    'classifier__penalty': ['l1', 'l2']
}

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform random search to find the best model parameters with fewer folds and iterations
random_search = RandomizedSearchCV(pipeline, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, n_iter=5, random_state=42)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Predict probabilities on the test set
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Evaluate the model using area under the ROC curve
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'Best model parameters: {random_search.best_params_}')
print(f'Area under the ROC curve: {roc_auc}')


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np

# Load the test1 dataframe and sample submission format
test1 = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e7/sample_submission.csv')
# Take a sample of the data to reduce size (adjust size as needed)
df_sample = df.sample(frac=0.1, random_state=42)  # 5% of the data

# Split the data into features and target (assuming 'Response' is the target column)
X = df_sample.drop(columns='Response')
y = df_sample['Response']

# Identify categorical and numerical columns
categorical_columns = X.select_dtypes(include=['object', 'category']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing pipeline for numerical data
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing pipeline for categorical data
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_columns),
        ('cat', categorical_pipeline, categorical_columns)
    ]
)

# Define the ensemble models
rf = RandomForestClassifier(random_state=42)
gbc = GradientBoostingClassifier(random_state=42)
abc = AdaBoostClassifier(random_state=42)

# Combine preprocessing steps and each model into a single pipeline
pipelines = {
    'RandomForest': Pipeline(steps=[('preprocessor', preprocessor), ('classifier', rf)]),
    'GradientBoosting': Pipeline(steps=[('preprocessor', preprocessor), ('classifier', gbc)]),
    'AdaBoost': Pipeline(steps=[('preprocessor', preprocessor), ('classifier', abc)])
}

# Define the parameter grid for random search for each model
param_grids = {
    'RandomForest': {
        'classifier__n_estimators': [100],
        'classifier__max_depth': [None, 10]
    },
    'GradientBoosting': {
        'classifier__n_estimators': [100],
        'classifier__learning_rate': [0.1]
    },
    'AdaBoost': {
        'classifier__n_estimators': [50],
        'classifier__learning_rate': [0.1]
    }
}

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform random search to find the best model parameters
best_estimators = {}
for model_name in pipelines:
    grid_search = RandomizedSearchCV(pipelines[model_name], param_grids[model_name], cv=3, scoring='roc_auc', n_jobs=-1, n_iter=2, random_state=42)
    grid_search.fit(X_train, y_train)
    best_estimators[model_name] = grid_search.best_estimator_
    print(f'Best parameters for {model_name}: {grid_search.best_params_}')

# Define the base models for stacking
base_models = [(name, model) for name, model in best_estimators.items()]

# Define the meta-model for stacking
meta_model = LogisticRegression(solver='liblinear', random_state=42)

# Create the stacking classifier
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=3, n_jobs=-1)

# Fit the stacking classifier
stacking_clf.fit(X_train, y_train)

# Predict probabilities on the test set and evaluate the models
roc_aucs = {}
for model_name, model in best_estimators.items():
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    roc_aucs[model_name] = roc_auc
    print(f'Area under the ROC curve for {model_name}: {roc_auc}')

# Evaluate the stacking classifier
y_pred_proba_stacking = stacking_clf.predict_proba(X_test)[:, 1]
roc_auc_stacking = roc_auc_score(y_test, y_pred_proba_stacking)
roc_aucs['StackingClassifier'] = roc_auc_stacking
print(f'Area under the ROC curve for StackingClassifier: {roc_auc_stacking}')

# Compare the models
best_model_name = max(roc_aucs, key=roc_aucs.get)
print(f'\nThe best model is {best_model_name} with an AUC of {roc_aucs[best_model_name]}')


In [ ]:
# Predict on test1 set
#X_test1 = test1.drop(columns='Response')
#X_test1_preprocessed = preprocessor.transform(test1)
predictions = stacking_clf.predict_proba(test1)[:, 1]

# Save predictions to a new CSV file following the sample_submission1 format
submission = sample_submission.copy()
submission['Response'] = predictions
submission.to_csv('submission.csv', index=False)
submission.head()